<a href="https://colab.research.google.com/github/adelyaahmadullina/carnaval/blob/main/text_to_video_synthesis_diffusers_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/huggingface/diffusers/pull/2738 modified
!pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install diffusers==0.19.0 transformers accelerate imageio[ffmpeg] -U
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
  Using cached nv

In [2]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode
import datetime
#генерируем модель
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Включение опции CPU-разгрузки модели для ускорения работы
pipe.enable_model_cpu_offload()

# Включение разрезания изображений методом VAE для обработки больших размеров видеофайлов
pipe.enable_vae_slicing()

# Создание каталога /content/videos для сохранения видеофайлов
!mkdir /content/videos


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [25]:
prompts = ['vintage carnival with painted wagons and twinkling lights',
           'vintage carnival atmosphere: painted wagons, twinkling lights, and realistic performers in vibrant costumes',
           'wagons adorned with painting, lights softly twinkling, and performers in vintage costumes creating an atmosphere of festivity',
           'carnival scene: vintage wagons, twinkling lights, and performers in vibrant costumes reminiscent of a bygone era',
           'bright costumes of performers, painted wagons, and twinkling lights creating the atmosphere of a vintage carnival',
           'vintage carnival wagons adorned with colorful paintings, illuminated by twinkling lights, and performers in vibrant costumes',
           'scene of a vintage carnival: festive atmosphere with painted wagons, twinkling lights, and performers in vibrant costumes',
           'retro carnival with vibrant performer costumes, painted wagons, and twinkling lights']


In [26]:
number = 0
while  number < len(prompts):
  prompt = prompts[number]
  negative_prompt = "low quality"
  num_frames = 20

  video_frames = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=25, num_frames=num_frames).frames

  output_video_path = export_to_video(video_frames)

  new_video_path = f'/content/videos/{datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}.mp4'
  #Преобразование в mp4 формат
  !ffmpeg -y -i {output_video_path} -c:v libx264 -c:a aac -strict -2 {new_video_path} >/dev/null 2>&1

  print(output_video_path, '->', new_video_path)
  number += 1



  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/tmpn4gk04kn.mp4 -> /content/videos/2024-05-10_09:41:41.mp4


  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/tmp0vmg3hxf.mp4 -> /content/videos/2024-05-10_09:42:16.mp4


  0%|          | 0/25 [00:00<?, ?it/s]

...

/tmp/tmpoettgjcc.mp4 -> /content/videos/2024-05-10_09:44:32.mp4


  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/tmppyc8rc11.mp4 -> /content/videos/2024-05-10_09:45:06.mp4


  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/tmp93u1je_o.mp4 -> /content/videos/2024-05-10_09:45:40.mp4


In [27]:
from moviepy.editor import *
import os

content = os.listdir('/content/videos')

print(content)


['2024-05-10_09:42:50.mp4', '2024-05-10_09:45:40.mp4', '2024-05-10_09:41:41.mp4', '2024-05-10_09:43:58.mp4', '2024-05-10_09:45:06.mp4', '2024-05-10_09:42:16.mp4', '2024-05-10_09:43:24.mp4', '2024-05-10_09:44:32.mp4']


In [28]:
list_video = []
number_video = 0



In [29]:
while number_video < len(content):
    clip = VideoFileClip(f"/content/videos/{content[number_video]}")
    number_video += 1
    list_video.append(clip)
result_clip = concatenate_videoclips(list_video)
result_clip.write_videofile('combined.mp4')


Moviepy - Building video combined.mp4.
Moviepy - Writing video combined.mp4



Moviepy - Done !
Moviepy - video ready combined.mp4


In [30]:
import os

# Указать путь к папке, из которой нужно удалить файлы
folder_path = '/content/videos/'

# Получить список всех файлов в указанной папке
files = os.listdir(folder_path)

# Пройтись по каждому файлу и удалить его
for file in files:
    file_path = os.path.join(folder_path, file)  # Формирование полного пути к файлу
    os.remove(file_path)  # Удаление файла

print("Файлы успешно удалены из папки 'videos'.")

Файлы успешно удалены из папки 'videos'.
